In [71]:
using DataFrames, DataFramesMeta, Optim, Revise, Serialization, DebuggingUtilities, LinearAlgebra, Statistics
using JuMP, Ipopt

In [11]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule; const m = BLPmodule;

In [12]:
test1state = true
if test1state 
    dirpath = "/export/storage_adgandhi/MiscLi/factract/analysis"
else
    dirpath = "jls"
end;
ec = deserialize("$dirpath/ec.jls");

---
### jump

In [69]:
# create some indices 
# t_indexer[jj] is the vector of markets (i.e. their indices) served by facility jj
t_indexer = [[] for _ in 1:length(ec.firms)]
for tt in eachindex(ec.tracts)
    for ind in ec.tracts[tt].inds
        push!(t_indexer[ind], tt)
    end
end

# positionin[jj,tt] is firm jj's position in tract tt's vectors (e.g. the vector of market shares in tt)
positionin = Matrix(undef, n_firms, n_tracts)
for jj in 1:n_firms
    for tt in t_indexer[jj]
        positionin[jj,tt] = findall(x->x == jj, j_indexer[tt])[1]
    end
end

In [108]:
n_firms = length(ec.firms);
n_firms_intract = [t.n_firms for t in ec.tracts]
n_tracts = length(ec.tracts);
nX = 2
nD = 1
X = [t.X[:,1:nX] for t in ec.tracts]
D = [t.D[:,1:nD] for t in ec.tracts]
j_indexer = [t.inds for t in ec.tracts]
M = [t.M for t in ec.tracts]
q_obs = ec.q_obs;

In [109]:
model = Model(Ipopt.Optimizer)
@variables(model, begin
    θx[1:nX]
    θd[1:nD]
end)

u = @NLexpression(model,
    [tt in 1:n_tracts, jj in j_indexer[tt]],
    sum(θx[ix] * X[tt][positionin[jj,tt],ix] for ix in 1:nX) + sum(θd[id] * D[tt][positionin[jj,tt],id] for id in 1:nD))

expu = @NLexpression(model,
    [tt in 1:n_tracts, jj in j_indexer[tt]],
    exp(u[tt,jj]))

denom = @NLexpression(model,
    [tt in 1:n_tracts],
    1+ sum(expu[tt,jj] for jj in j_indexer[tt]))

share = @NLexpression(model,
    [tt in 1:n_tracts, jj in j_indexer[tt]],
    expu[tt,jj] / denom[tt])

mktq = @NLexpression(model,
    [tt in 1:n_tracts, jj in j_indexer[tt]],
    M[tt] * share[tt,jj])

firmq = @NLexpression(model,
    [jj in 1:n_firms],
    sum(mktq[tt, jj] for tt in t_indexer[jj]))

@NLobjective(model, Min, sum((firmq[jj] - q_obs[jj])^2 for jj in 1:n_firms))

optimize!(model)


This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.4612031e+08 0.00e+00 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

In [110]:
value.(θx)

2-element Vector{Float64}:
 -0.4476982868894854
 -0.19014636125654918

In [111]:
value.(θd)

1-element Vector{Float64}:
 -43.78729907909843

In [112]:
q_preds = value.(firmq);
share_preds = value.(share);
# index with share_preds[tt,jj]

JuMP.Containers.SparseAxisArray{Float64, 2, Tuple{Int64, Int64}} with 55472 entries:
  [1037, 337]  =  3.91075e-5
  [1134, 151]  =  0.103185
  [1214, 329]  =  0.000156175
  [1243, 12 ]  =  0.0035972
  [1332, 208]  =  0.00286307
  [1410, 164]  =  0.000989789
  [1674, 383]  =  5.52232e-5
  [1778, 391]  =  0.000474961
               ⋮
  [2291, 38 ]  =  0.00406708
  [2355, 40 ]  =  0.00056669
  [308, 163 ]  =  4.03684e-5
  [457, 232 ]  =  0.000473442
  [498, 222 ]  =  0.00600919
  [620, 295 ]  =  0.00348861
  [734, 375 ]  =  0.00194472

In [116]:
θD1 = value.(θd);
# θD1, θD2 = value.(θd);

In [117]:
ηDj = [sum([share_preds[tt,jj]*M[tt]*(1-share_preds[tt,jj])*(θD2*D[tt][positionin[jj,tt],1])*D[tt][positionin[jj,tt],1] for tt in t_indexer[jj]]) / q_preds[jj] for jj in 1:n_firms]

410-element Vector{Float64}:
  0.8809502220429122
  0.845402291137059
  0.8288421368700702
  0.5616987130547682
  0.8917171872500751
  0.46951478346557163
  1.134575477035751
  1.0414943759429935
  0.8260159016430056
  1.1190469548121327
  ⋮
  0.6612041576108612
  1.2727885167223525
  1.3982544722444579
 10.335507327156156
  1.1557192850242373
  8.967496340864116
  0.7360138536748763
  1.1357190036318712
  0.820450319280087

In [ ]:
ηDj = [sum([share_preds[tt,jj]*M[tt]*(1-share_preds[tt,jj])*(θD2*D[tt][positionin[jj,tt],1])*D[tt][positionin[jj,tt],1] for tt in t_indexer[jj]]) / q_preds[jj] for jj in 1:n_firms]

410-element Vector{Float64}:
  0.8809502220429122
  0.845402291137059
  0.8288421368700702
  0.5616987130547682
  0.8917171872500751
  0.46951478346557163
  1.134575477035751
  1.0414943759429935
  0.8260159016430056
  1.1190469548121327
  ⋮
  0.6612041576108612
  1.2727885167223525
  1.3982544722444579
 10.335507327156156
  1.1557192850242373
  8.967496340864116
  0.7360138536748763
  1.1357190036318712
  0.820450319280087

In [118]:
mean(ηDj)

0.9727332061959705